# Obtaining info from Google sheets

In this notebook we are going to obtain info recopilated in google sheets of some manual analysis, which comes in handy for notebooks such as **7HM**. 
To do that we use the python API from Google sheets, and store the necessary information in variables that will be loaded in the respective notebooks.

Therefore **load this notebook before loading any other notebooks in the analysis**.

To load the notebook you may need to have a Google account. If you have any trouble in that process, contact me (@alexmascension in GitHub).
Nonetheless, the parts of the notebooks that require these variables are not required for the analysis directly, but they come in handy to me to set information about some genes that, otherwise, would require a more lengthy searching process at the google sheets directly.

In [ ]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1lfI6sgjEyg37BGL7VRMfW7KgwGKwX5QrCtnKYk1DXY4'
SAMPLE_RANGE_NAME_HUMAN = 'Gene info (Human)!A:C'
SAMPLE_RANGE_NAME_MOUSE = 'Gene info (Mouse)!A:C'
JSON_FILE = "data/TOKENS/client_secret_606804245862-te1c02pcqrhb9n298v5ja9as7r9qk5li.apps.googleusercontent.com.json"

In [ ]:
# GET CREDS
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            JSON_FILE, SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

In [ ]:
service = build('sheets', 'v4', credentials=creds)

# Call the Sheets API
sheet = service.spreadsheets()
result_human = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range=SAMPLE_RANGE_NAME_HUMAN).execute()

result_mouse = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range=SAMPLE_RANGE_NAME_MOUSE).execute()

In [ ]:
df_human_genes_codes = pd.DataFrame({'gene': [i[0] for i in result_human['values'][1:]], 'code': [i[2] for i in result_human['values'][1:]]}).set_index('gene')
df_human_genes_codes

In [ ]:
df_mouse_genes_codes = pd.DataFrame({'gene': [i[0] for i in result_mouse['values'][1:]], 'code': [i[2] for i in result_mouse['values'][1:]]}).set_index('gene')
df_mouse_genes_codes

In [ ]:
%store df_human_genes_codes
%store df_mouse_genes_codes

In [ ]:
os.system('jupyter nbconvert --to html 0A_obtaining_Google_drive_sheets_info.ipynb')